# Deep Q-Network (DQN)

**Description:** Implementing DQN algorithm on the Robot Navigation Problem.

## Introduction

**Deep Q-Network (DQN)** is a model-free off-policy algorithm for learning discrete actions.

It uses neural networks to apply function approximation, to estimate the Q-function while avoiding convergence problems in the case of continous actions space, states space or both.

It uses a method called replayed memory, which represents a memory buffer for storing the experiences of the agent.

## Problem

We are trying to solve the **Robot Navigation** problem.
In this setting, we can assume taking three actions: [Forward, LeftTurn, RightTurn].


The implementation of the algorithm is carried out in the rest of the file.

In [33]:
import numpy as np
import gym
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mean_squared_error
from matplotlib import pyplot as plt

In [ ]:

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.n_actions = action_size
        '''
        # "lr" : learning rate
        # "gamma": discounted factor
        # "exploration_proba_decay": decay of the exploration probability
        # "batch_size": size of experiences we sample to train the DNN
        '''
        self.lr = 0.001
        self.gamma = 0.99
        self.exploration_proba = 1.0
        self.exploration_proba_decay = 0.005
        self.batch_size = 32
        
        # We define our memory buffer where we will store our experiences
        # We stores only the 2000 last time steps
        self.memory_buffer= list()
        self.max_memory_buffer = 2000
        
        # We create our model having to hidden layers of 24 units (neurones)
        # The first layer has the same size as a state size
        # The last layer has the size of actions space
        self.model = Sequential([
            Dense(units=24,input_dim=state_size, activation = 'relu'),
            Dense(units=24,activation = 'relu'),
            Dense(units=action_size, activation = 'linear')
        ])
        self.model.compile(loss="mse",
                      optimizer = Adam(lr=self.lr))